<a href="https://colab.research.google.com/github/allen651212/DeepFaceLab2_Colab/blob/master/DeepFaceLab2_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

此脚本由Allen651212创建！

https://github.com/allen651212/DeepFaceLab2_Colab 


In [4]:
#@title 查看分配到的设备
# 查看设备，是K80还是T4,如果是K80...！
! /opt/bin/nvidia-smi

Tue Feb 11 11:57:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# 第一步 准备好workspace
这一步你可以选着两种方式
1. 使用默认的workspace，你无需自己上传，仅用于熟悉操作。
2. 通过Google Drive （谷歌云盘）上传自己的workspace到指定目录。



谷歌云盘地址：https://drive.google.com/drive/my-drive  
注意：谷歌网址现在都需要“科学上网”才能访问。  

In [0]:
#@title 开始挂载 
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#创建DeepFaceLab目录，并且进入目录
%cd /content/drive/My Drive/
!mkdir DeepFaceLab2
%cd /content/drive/My Drive/DeepFaceLab2/


这一环节有两个选择：  
如果你第一次使用，可以加载示例项目，然后往后运行。  
如果你是二次运行，就不需要加载了。  
如果你希望上传本地素材，可以先用pack脚本把素材打包成faceset.pak到aligned目录，无须解压，可直接进行训练和转换，加载速度会很快。


In [0]:
#@title 加载示例项目workspace 
# 1. 练手可以使用这一行，直接git clone一个workspace
%cd /content/drive/My Drive/DeepFaceLab2/
!rm -rf workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

#第二步 安装DeepFaceLab
获取源代码，安装依赖,根据自己的情况选择版本  
  
last:最新版  

In [0]:
#@title 开始安装

Version = "last" #@param ["last"]
%cd /content/drive/My Drive/DeepFaceLab2
!rm -fr DeepFaceLab2_Colab
if Version=="last":
  print("最新版加载中....")
  !git clone https://github.com/allen651212/DeepFaceLab2_Colab
  %cd /content/drive/My Drive/DeepFaceLab2/DeepFaceLab2_Colab
  !pip install -r requirements-colab.txt

!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0


# 第三步. 提取脸部

这个步骤其实是分两次的。第一次选src，第二次选dst，很多对Deepfacelab不了解的人可能只点了一个，后面就会报错了。  
  
这一步，其实有三个小步骤，分解视频，提取人脸，打包素材。 如果有需要可以将他们拆分开来。一般是不用这么做的。 





In [0]:
#@title 开始提取
target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab2/DeepFaceLab2_Colab
if target=="src":
  !python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
  !python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
  !python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
  !python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug
  !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset


In [0]:
#@title 开始排序
target = "src" #@param ["src","dst"]
order = "hist" #@param ["blur", "face", "face-dissim", "face-yaw", "face-pitch", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final", "final-no-blur"]

if target=="src":
  #Src排序，可以通过谷歌云盘查看结果，删除不良图片
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned --by "+order
  
else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned --by "+order

!python $cmd

# 第四步. 训练模型


*  支持SAEHD,Quick96等模型，根据自己的情况选择模型。
*  第一次需要配置模型参数，不懂的直接回车默认。
*  不想训练了可以点击停止，停止时会抛出异常，但是没什么关系。下次可以继续训练

In [0]:
#@title 开始训练
Model = "SAEHD" #@param ["SAEHD", "Quick96"]

%cd "/content/drive/My Drive/DeepFaceLab2/DeepFaceLab2_Colab/"
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

**模型预览方法请参考：**
https://www.wxp123.me/p/329

简单描述：左侧->文件->/driver/..../model/ *preview*.jpg->双击  
  
这种方法比在脚本中预览和在网盘预览都要方便，并且无需启动预览配置！

# 第五步. 转换输出

训练模型的时候选了什么，这里就选什么  
比如训练SAEHD,这里就选SAEHD


In [0]:
#@title 开始转换图片

Model = "SAEHD" #@param ["SAEHD", "Quick96"]
%cd "/content/drive/My Drive/DeepFaceLab2/DeepFaceLab2_Colab/"
cmd = " main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

In [0]:
#@title 开始图片转视频
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

#第六步. 继续训练
当你第二次开始训练，或者掉线之后继续训练时并不需要执行上面所有的步骤。只需要下面简单的几个步骤。
1. 挂载云盘
2. 安装依赖
3. 开始训练  


In [0]:
#@title 一键运行
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

Model = "SAEHD" #@param ["SAEHD", "Quick96"]


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#重新覆盖解压，提升加载速度，可节省很多时间
%cd /content/drive/My Drive/DeepFaceLab2/
!unzip -o workspace.zip


# 进入DeepFaceLab2_Colab目录
%cd /content/drive/My Drive/DeepFaceLab2/DeepFaceLab2_Colab

# 安装Python依赖
!pip install -r requirements-colab.txt
!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0

# 开始训练SAEHD ，如果是其他模型，修改后面的参数即可，比如Quick96。
cmd ="main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model +" --no-preview"
!python $cmd 

# 工具



打包素材。可以加快上传，下载，加载的速度。

In [0]:
#@title 素材打包

target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab2/DeepFaceLab2_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
 !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset
 
  

In [0]:
#@title 素材解包

target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab2/DeepFaceLab2_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned --unpack-faceset
else:
 !python main.py util --input-dir ../workspace/data_dst/aligned --unpack-faceset
 
  

In [0]:
#@title 素材增强

target = "src" #@param ["src"]
%cd /content/drive/My Drive/DeepFaceLab2/DeepFaceLab2_Colab
if target=="src":
  !python main.py facesettool enhance --input-dir ../workspace/data_src/aligned
else:
  !python main.py facesettool enhance --input-dir ../workspace/data_dst/aligned
 
  